In [1]:
from ultralytics import YOLO
import torch
import torch.nn as nn

# 커스텀 헤드 정의
class CustomHead(nn.Module):
    def __init__(self, nc=80, ch=()):
        super().__init__()
        self.nc = nc
        
        self.conv1 = nn.Conv2d(ch[0], 256, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        
        self.cls_conv = nn.Conv2d(256, self.nc, 1)
        self.reg_conv = nn.Conv2d(256, 4, 1)
        self.obj_conv = nn.Conv2d(256, 1, 1)
        
        self.act = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.act(self.bn1(self.conv1(x)))
        x = self.act(self.bn2(self.conv2(x)))
        
        cls_out = self.cls_conv(x)
        reg_out = self.reg_conv(x)
        obj_out = self.obj_conv(x)
        
        return torch.cat([reg_out, obj_out, cls_out], 1)

# 커스텀 로스 정의
class CustomLoss:
    def __init__(self):
        self.bce = nn.BCEWithLogitsLoss()
        self.l1 = nn.L1Loss()
        
    def __call__(self, preds, targets):
        cls_loss = self.bce(preds[..., 5:], targets[..., 5:])
        box_loss = self.l1(preds[..., :4], targets[..., :4])
        obj_loss = self.bce(preds[..., 4], targets[..., 4])
        return cls_loss + box_loss + obj_loss

# 모델 설정
model = YOLO('yolo11n.pt')
model.model.head = CustomHead(nc=80, ch=[256])
model.loss_fn = CustomLoss()

# # 학습 설정
# data_yaml = """
# path: ../datasets/test  # 데이터셋 루트 디렉토리
# train: images/train  # 학습 이미지 경로
# val: images/val  # 검증 이미지 경로

# nc: 80  # 클래스 수
# names: ['person', 'bicycle', ...]  # 클래스 이름들
# """

# # data.yaml 파일 생성
# with open('data.yaml', 'w') as f:
#     f.write(data_yaml)

# 학습 실행
results = model.train(
    data='coco8.yaml',
    epochs=3,               # 빠른 테스트를 위해 3 에포크만
    imgsz=640,             # 이미지 크기
    batch=16,              # 배치 크기
    save=True,             # 모델 저장
    device='cuda',         # GPU 사용
    project='yolo_test',   # 프로젝트 이름
    name='custom_head',    # 실험 이름
)

# 학습 결과 확인
print("Training completed!")
print(f"Best mAP50: {results.maps[50]}")
# print(f"Final model saved at: {results.model}")

# 테스트 추론
test_img = torch.randn(1, 3, 640, 640)  # 테스트 이미지
results = model(test_img)
print("\nTest inference results:")
for r in results:
    print(f"Detected objects: {len(r.boxes)}")
    if r.boxes:
        print(f"Confidence scores: {r.boxes.conf}")

New https://pypi.org/project/ultralytics/8.3.28 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.17  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=coco8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=yolo_test, name=custom_head3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning D:\Workspace\SatellieteIMGDetection\datasets\coco8\labels\train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


c:\Users\pook0\.conda\envs\UM\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning D:\Workspace\SatellieteIMGDetection\datasets\coco8\labels\val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


Plotting labels to yolo_test\custom_head3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_test\custom_head3
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        3/3     0.696G      1.436      3.954      1.814         15        640: 100%|██████████| 1/1 [00:00<00:00,  8.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 21.28it/s]

                   all          4         17      0.577       0.85      0.853       0.63



3 epochs completed in 0.001 hours.
Optimizer stripped from yolo_test\custom_head3\weights\last.pt, 5.5MB
Optimizer stripped from yolo_test\custom_head3\weights\best.pt, 5.5MB

Validating yolo_test\custom_head3\weights\best.pt...
Ultralytics 8.3.17  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16380MiB)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 25.96it/s]


                   all          4         17      0.581       0.85      0.851      0.616
                person          3         10      0.569        0.6      0.605      0.275
                   dog          1          1      0.546          1      0.995      0.697
                 horse          1          2      0.584          1      0.995      0.674
              elephant          1          2      0.365        0.5      0.524      0.259
              umbrella          1          1      0.569          1      0.995      0.895
          potted plant          1          1      0.851          1      0.995      0.895
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to yolo_test\custom_head3
Training completed!
Best mAP50: 0.615894956390835

WARNING  torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.706320762634277. Dividing input by 255.
0: 640x640 (no detections), 33.1ms
Speed: 8.0ms preprocess, 33.1ms inference, 6.0ms pos

In [2]:
from ultralytics import YOLO

import torch
from ultralytics.nn.modules.block import C3k2

# 입력 텐서 생성 (배치 크기: 1, 채널 수: 64, 높이: 128, 너비: 128)
x = torch.randn(1, 64, 128, 128)

# C3k2 클래스 인스턴스 생성
# 입력 채널 수: 64, 출력 채널 수: 128, 블록 수: 3, c3k 사용 여부: True
model = C3k2(c1=64, c2=128, n=3, c3k=True)

# 모델에 입력 텐서 전달
output = model(x)

# 출력 텐서의 크기 출력
print(output.shape)

torch.Size([1, 128, 128, 128])


In [4]:
import torch
from ultralytics.nn.modules.conv import Conv

# 입력 텐서 생성 (배치 크기: 1, 채널 수: 3, 높이: 224, 너비: 224)
x = torch.randn(1, 30, 224, 224)

# Conv 클래스 인스턴스 생성
# 입력 채널 수: 3, 출력 채널 수: 64, 커널 크기: 3, 스트라이드: 1, 패딩: 자동
conv_layer = Conv(c1=30, c2=64, k=3, s=1)

# Conv 레이어에 입력 텐서 전달
output = conv_layer(x)

# 출력 텐서의 크기 출력
print(output.shape)

torch.Size([1, 64, 224, 224])


In [ ]:
import torch
from ultralytics.nn.modules.block import SPPF

'''
SPPF(Spatial Pyramid Pooling - Fast) 모듈은 YOLOv5에서 사용되는 레이어로, 입력 특징 맵에 대해 공간 피라미드 풀링을 수행하여 다양한 크기의 맥스 풀링 결과를 결합합니다. 이를 통해 다양한 스케일의 정보를 효과적으로 통합할 수 있습니다. SPPF 모듈은 특히 빠른 연산을 위해 최적화되어 있습니다.

주요 역할:

다양한 스케일의 정보 통합: 서로 다른 크기의 맥스 풀링을 통해 다양한 스케일의 정보를 추출하고 이를 결합합니다.
특징 맵의 크기 유지: 입력 특징 맵의 공간적 크기를 유지하면서 채널 수를 증가시킵니다.
연산 효율성: 빠른 연산을 위해 최적화된 구조를 사용합니다.
SPPF 모듈의 구조는 다음과 같습니다:

입력 채널 수를 절반으로 줄이는 1x1 컨볼루션 레이어
5x5 맥스 풀링 레이어를 세 번 반복하여 다양한 스케일의 정보를 추출
추출된 정보를 결합한 후 출력 채널 수를 원하는 값으로 조정하는 1x1 컨볼루션 레이어
이를 통해 입력 특징 맵의 공간적 크기를 유지하면서 다양한 스케일의 정보를 효과적으로 통합할 수 있습니다.
'''
# 입력 텐서 생성 (배치 크기: 1, 채널 수: 512, 높이: 32, 너비: 32)
x = torch.randn(1, 512, 32, 32)

# SPPF 클래스 인스턴스 생성
# 입력 채널 수: 512, 출력 채널 수: 1024, 커널 크기: 5
sppf_layer = SPPF(c1=512, c2=1024, k=5)

# SPPF 레이어에 입력 텐서 전달
output = sppf_layer(x)

# 출력 텐서의 크기 출력
print(output.shape)

torch.Size([1, 1024, 32, 32])


In [5]:
import torch
from ultralytics.nn.modules.block import C2PSA

# 입력 텐서 생성 (배치 크기: 1, 채널 수: 256, 높이: 64, 너비: 64)
x = torch.randn(1, 256, 64, 64)

# C2PSA 클래스 인스턴스 생성
# 입력 채널 수: 256, 출력 채널 수: 256, 블록 수: 3, 확장 비율: 0.5
model = C2PSA(c1=256, c2=256, n=3, e=0.5)

# 모델에 입력 텐서 전달
output = model(x)

# 출력 텐서의 크기 출력
print(output.shape)

torch.Size([1, 256, 64, 64])


In [32]:
import torch
from ultralytics.nn.modules.head import Detect

# 3개의 서로 다른 크기의 특징맵 생성 
# (batch_size, channels, height, width)
feature_maps = [
    torch.randn(1, 256, 80, 80),    # P3
    torch.randn(1, 256, 40, 40),    # P4  
    torch.randn(1, 256, 20, 20)     # P5
]

# Detect 레이어 초기화
# nc: 클래스 수 80개 (COCO dataset)
# anchors: 3개의 스케일(P3,P4,P5)에 각각 3개의 앵커
# ch: 입력 특징맵의 채널 수 
detect = Detect(nc=10, ch=[256, 256, 256])

# 추론 모드로 설정
detect.eval()

# 추론 실행
with torch.no_grad():
    predictions = detect(feature_maps)

# 출력 형태 확인 
# (batch_size, num_anchors_total, num_classes + 5)
print(predictions[0].shape)


torch.Size([1, 14, 8400])


In [31]:
temp = predictions[1]
print(temp[0].shape)
print(temp[1].shape)
print(temp[2].shape)


torch.Size([1, 144, 80, 80])
torch.Size([1, 144, 40, 40])
torch.Size([1, 144, 20, 20])
